# Simple ANN approach without any fancy stuff
## Upvote is the best motivator 👍

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
print(tf.__version__)
from sklearn.preprocessing import StandardScaler

In [ ]:
# Get data and use StandardScaler on it
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")
X = df_train[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']].to_numpy()

filter_col = [col for col in df_train if col.startswith('target')]
y = df_train[filter_col].to_numpy()

X_scaler = StandardScaler()
X = X_scaler.fit_transform(X)

y_scaler = StandardScaler()
y = y_scaler.fit_transform(y)

In [ ]:
# Create Neural Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=[8]),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(3)
])

# Compile
model.compile(loss="mse", optimizer='adam', metrics=["mae"])

# Train
history = model.fit(X, y, epochs=500, verbose=1)

In [ ]:
# Get Test data, use StandardScaler, predict and inverse Transform results
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")
X_test = df_test[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']].to_numpy()
X_test = X_scaler.transform(X_test)
y_test = model.predict(X_test)
y_test = y_scaler.inverse_transform(y_test)

# Create submission file
submission = pd.DataFrame(y_test, columns=[filter_col])
submission.insert(0, 'date_time', df_test['date_time'])
submission.to_csv('submission.csv', index=False)